# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
from pymystem3 import Mystem
m = Mystem()
lemmas = m.lemmatize(df['purpose'][1])
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод** Мы получили от банка данные статистики по клиентам, которые включают следующую информацию: наличие детей, рабочий стаж в днях, возраст, образование, семейный статус, пол, тип занятости, ежемесячный доход и цели кредита.
При ознакомлении с данными в глаза сразу бросается, что имеются неточности в передаче данных. Есть отрицательные значения по стажу, образование может быть написано заглавными и строчными буквами, есть пропуски по стажу и доходу, похожие цели записаны разными формулировками.
Прежде чем займемся анализом, необходмо подготовить данные.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
df['children'].value_counts() #Пропущенных значений нет, видим вероятные ошибки, исправим их чуть позже


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [4]:
len(df[df['days_employed'].isna()]) #Пропущенных значений 2174, заменим их на среднее по столбцу

2174

In [5]:
#Но сначала разберемся с очень большими значениями, для этого сделаем выборку по занятости и отработанным дням. 
#Найдем медиану, минимальное и максимальное значение
#У безработных и пенсионеров данные по стажу ошибочные, причина ошибки неявная. Заменим стаж на 0 по этим группам.
#У всех остальных отрицательные значения, здесь верояная причина ошибки - при заполнении было использовано тире. Просто переведем их в положительные.

In [6]:
income_type_median = df.groupby('income_type')['days_employed'].median() 
income_type_median

income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -2689.368353
компаньон           -1547.382223
пенсионер          365213.306266
предприниматель      -520.848083
сотрудник           -1574.202821
студент              -578.751554
Name: days_employed, dtype: float64

In [7]:
income_type_max = df.groupby('income_type')['days_employed'].max()
income_type_max

income_type
безработный        395302.838654
в декрете           -3296.759962
госслужащий           -39.954170
компаньон             -30.195337
пенсионер          401755.400475
предприниматель      -520.848083
сотрудник             -24.141633
студент              -578.751554
Name: days_employed, dtype: float64

In [8]:
income_type_min = df.groupby('income_type')['days_employed'].min()
income_type_min

income_type
безработный        337524.466835
в декрете           -3296.759962
госслужащий        -15193.032201
компаньон          -17615.563266
пенсионер          328728.720605
предприниматель      -520.848083
сотрудник          -18388.949901
студент              -578.751554
Name: days_employed, dtype: float64

In [9]:
#нужно все положительные сделать нулями

In [10]:
def days_employed_tru(days_employed):
    if days_employed > 0:
        return 0
    else:
        return days_employed
df['days_employed_true'] = df['days_employed'].apply(days_employed_tru)
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_true
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,-8437.673028
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,-4024.803754
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,-5623.422610
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,-4124.747207
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,0.000000
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,-926.185831
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,-2879.202052
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,-152.779569
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,-6929.865299
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,-2188.756445


In [11]:
df['days_employed_true'] = abs(df['days_employed_true']) #Сначала изменим отрицательные значения на положительные
days_employed_true_mean = df['days_employed_true'].mean() #Найдем среднее по столбцу со стажем
days_employed_true_mean

1934.1156226744938

In [12]:
df['days_employed_true'] = df['days_employed_true'].fillna(value=df['days_employed_true'].mean()) # Заменим пропуски на среднее значение

In [13]:
len(df[df['days_employed_true'].isna()])  #Проверим, пропущенных значений нет

0

In [14]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_true
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,8437.673028
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,4024.803754
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,5623.422610
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,4124.747207
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,0.000000
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,926.185831
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,2879.202052
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,152.779569
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,6929.865299
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,2188.756445


In [15]:
df['dob_years'].value_counts() #Пропущенных значений нет, есть значения равные 0.

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [16]:
# нужно избавиться от нулей в возрасте, заменим на медианное значение
df.loc[df['dob_years'] == 0, 'dob_years'] = df['dob_years'].median()
df['dob_years'].value_counts()

42.0    698
35.0    617
40.0    609
41.0    607
34.0    603
38.0    598
33.0    581
39.0    573
31.0    560
36.0    555
44.0    547
29.0    545
30.0    540
48.0    538
37.0    537
50.0    514
43.0    513
32.0    510
49.0    508
28.0    503
45.0    497
27.0    493
56.0    487
52.0    484
47.0    480
54.0    479
46.0    475
58.0    461
57.0    460
53.0    459
51.0    448
59.0    444
55.0    443
26.0    408
60.0    377
25.0    357
61.0    355
62.0    352
63.0    269
64.0    265
24.0    264
23.0    254
65.0    194
66.0    183
22.0    183
67.0    167
21.0    111
68.0     99
69.0     85
70.0     65
71.0     58
20.0     51
72.0     33
19.0     14
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

In [17]:
df['education'].value_counts() #Пропущенных значений нет

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [18]:
df['education_id'].value_counts() #Пропущенных значений нет

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [19]:
df['family_status'].value_counts() #Пропущенных значений нет

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [20]:
df['family_status_id'].value_counts() #Пропущенных значений нет

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [21]:
df['gender'].value_counts() #Пропущенных значений нет, есть 1 незаполненное значение по полу. Оставим его без изменений

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [22]:
df['income_type'].value_counts() #Пропущенных значений нет

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [23]:
len(df[df['total_income'].isna()]) #Пропущенных значений 2174, заменим их на медианное значение по доходу

2174

In [24]:
total_income_median = df['total_income'].median()
total_income_median

145017.93753253992

In [25]:
total_income_mean = df['total_income'].mean() #Заодно найдем среднее, мин и максимум, пригодится для дальнейщего анализа
total_income_mean

167422.30220817294

In [26]:
total_income_max = df['total_income'].max()
total_income_max

2265604.028722744

In [27]:
total_income_min = df['total_income'].min()
total_income_min

20667.26379327158

In [28]:
df['total_income'] = df['total_income'].fillna(value=df['total_income'].median())
len(df[df['total_income'].isna()])

0

In [29]:
df['purpose'].value_counts() #Пропущенных значений нет

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод** При анализе данных обнаружили, что категории "пенсионеры" и "безработные" в столбце со стажем содержат неверную информацию, данные очень большие, в реальности таких быть не может. Такие занчения мы заменили на "0". По всем остальным категориям столбца "стаж" значения были отрицательными. Предполагем, что была ошибка при заполнении данных - перед цифрой ставили "тире". Отрицательные значения перевели в положительные. Пропущенные значения в этом столбце заменили на среднее.
В столбце с возврастом пропущенных значений не было, но были значения равные "0". Мы их заменили на медианное значение по этому столбцу.
Аналогично поступили с данными в столбце "доход", пропущенные значения заменили на медианное значение.



### Замена типа данных

In [30]:
df.info() #Видим, что в днях стажа и дохода нецелые числа, это будет неудобно для рассчетов

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children              21525 non-null int64
days_employed         19351 non-null float64
dob_years             21525 non-null float64
education             21525 non-null object
education_id          21525 non-null int64
family_status         21525 non-null object
family_status_id      21525 non-null int64
gender                21525 non-null object
income_type           21525 non-null object
debt                  21525 non-null int64
total_income          21525 non-null float64
purpose               21525 non-null object
days_employed_true    21525 non-null float64
dtypes: float64(4), int64(4), object(5)
memory usage: 2.1+ MB


In [31]:
df['days_employed_true'] = df['days_employed_true'].astype('int') #Переведем данные по стажу и доходу к целым числам
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children              21525 non-null int64
days_employed         19351 non-null float64
dob_years             21525 non-null float64
education             21525 non-null object
education_id          21525 non-null int64
family_status         21525 non-null object
family_status_id      21525 non-null int64
gender                21525 non-null object
income_type           21525 non-null object
debt                  21525 non-null int64
total_income          21525 non-null int64
purpose               21525 non-null object
days_employed_true    21525 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


**Вывод** Выясняем тип данных по столбцам. Видим, что стаж в днях и доход указаны дробными числами. Для удобства переводим их в целые.

### Обработка дубликатов

In [32]:
df['children'] = df['children'].replace(-1, 1) #Минус 1 ребенок, была очевидно допущена при заполнении данных, поменяем на 1 ребенка
df['children'] = df['children'].replace(20, df['children'].median()) #Вероятную ошибку в "20 детей", заменим на медианное значение
df['children'].value_counts()

0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [33]:
df['education'] = df['education'].str.lower() #Приведем все записи к одному регистру
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [34]:
df['family_status'] = df['family_status'].str.lower() #Приведем все записи к одному регистру
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [35]:
df.duplicated().sum()

72

In [36]:
df.drop_duplicates(inplace = True) 
df.duplicated().sum()

0

In [37]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_true
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,8437
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,4024
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5623
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4124
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,0
5,0,-926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,926
6,0,-2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,2879
7,0,-152.779569,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,152
8,2,-6929.865299,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,6929
9,0,-2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,2188


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21524
Data columns (total 13 columns):
children              21453 non-null int64
days_employed         19351 non-null float64
dob_years             21453 non-null float64
education             21453 non-null object
education_id          21453 non-null int64
family_status         21453 non-null object
family_status_id      21453 non-null int64
gender                21453 non-null object
income_type           21453 non-null object
debt                  21453 non-null int64
total_income          21453 non-null int64
purpose               21453 non-null object
days_employed_true    21453 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 2.3+ MB


**Вывод** Данные по столбцам "образование" и "семейный статус" приводим к единому нижнему регистру, стобы убрать дубли.
Данные по столбцу с детьми также приводим в порядок. Были значения "-1", предлогаем, что ошибка была допущена при заполнении и поставлено "тире" перед 1.
Такие значения меняем на 1. Заодно исправляем ошибку в данных по столбцу "дети" со значением 20 на медианное по столбцу. 
По этому значению ошибка могла быть допущена и при указании 2 детей и 0 детей.
Удалили полностью повторяющиеся строки.

### Лемматизация

In [39]:
def lemm_df(string):
    return m.lemmatize(string)
df['purpose_lemm'] = df['purpose'].apply(lemm_df)


In [40]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_true,purpose_lemm
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,8437,"[покупка, , жилье, \n]"
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,4024,"[приобретение, , автомобиль, \n]"
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5623,"[покупка, , жилье, \n]"
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4124,"[дополнительный, , образование, \n]"
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,0,"[сыграть, , свадьба, \n]"
5,0,-926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,926,"[покупка, , жилье, \n]"
6,0,-2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,2879,"[операция, , с, , жилье, \n]"
7,0,-152.779569,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,152,"[образование, \n]"
8,2,-6929.865299,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,6929,"[на, , проведение, , свадьба, \n]"
9,0,-2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,2188,"[покупка, , жилье, , для, , семья, \n]"


**Вывод** Цели кредита могут быть одинаковыми, но в тоже время записаны по разному. Для того чтобы в дальнейшем мы могли четко выделить эти цели, создаем столбец с лемматизированными словами из изначального.

### Категоризация данных

In [41]:
def purp_loa(purpose_lemm):
    if "автомобиль" in purpose_lemm:
        return "авто"
    elif "свадьба" in purpose_lemm:
        return "свадьба"
    elif "недвижимость" in purpose_lemm:
        return "недвижимость"
    elif "жилье" in purpose_lemm:
        return "недвижимость"
    else:
        return "образование"
df['purpose_loan'] = df['purpose_lemm'].apply(purp_loa)
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_true,purpose_lemm,purpose_loan
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,8437,"[покупка, , жилье, \n]",недвижимость
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,4024,"[приобретение, , автомобиль, \n]",авто
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5623,"[покупка, , жилье, \n]",недвижимость
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4124,"[дополнительный, , образование, \n]",образование
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,0,"[сыграть, , свадьба, \n]",свадьба
5,0,-926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,926,"[покупка, , жилье, \n]",недвижимость
6,0,-2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,2879,"[операция, , с, , жилье, \n]",недвижимость
7,0,-152.779569,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,152,"[образование, \n]",образование
8,2,-6929.865299,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,6929,"[на, , проведение, , свадьба, \n]",свадьба
9,0,-2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,2188,"[покупка, , жилье, , для, , семья, \n]",недвижимость


In [42]:
def total_income_stat(total_income):
    if total_income <= 50000:
        return 'низкий'
    elif total_income > 50000 and total_income <= 130000:
        return 'средний'
    elif total_income > 130000 and total_income <= 250000:
        return 'выше среднего'
    else:
        return 'высокий'
df['total_income_status'] = df['total_income'].apply(total_income_stat)
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_true,purpose_lemm,purpose_loan,total_income_status
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,8437,"[покупка, , жилье, \n]",недвижимость,высокий
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,4024,"[приобретение, , автомобиль, \n]",авто,средний
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5623,"[покупка, , жилье, \n]",недвижимость,выше среднего
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4124,"[дополнительный, , образование, \n]",образование,высокий
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,0,"[сыграть, , свадьба, \n]",свадьба,выше среднего
5,0,-926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,926,"[покупка, , жилье, \n]",недвижимость,высокий
6,0,-2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,2879,"[операция, , с, , жилье, \n]",недвижимость,выше среднего
7,0,-152.779569,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,152,"[образование, \n]",образование,выше среднего
8,2,-6929.865299,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,6929,"[на, , проведение, , свадьба, \n]",свадьба,средний
9,0,-2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,2188,"[покупка, , жилье, , для, , семья, \n]",недвижимость,выше среднего


**Вывод** Выделили основные цели кредита - образование, свадьба, автомобиль и недвижимость. Через создание нового столбца присвоили одну из категорий, но основании леммы.
Раздели доход также на категории - низкий, средний, выше среднего и высокий. В зависимости от размера дохода каждому значению в новом столбце присвоили свою категорию дохода.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [43]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = df.groupby('children')['debt'].count()
debt_from_children['sum_children'] = df.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] 
debt_from_children

,count_children,sum_children,result_children
children,,,
0,14166,1071,0.075604
1,4855,445,0.091658
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод** Бездетные самые дисциплинированные. Заемщики с 3 детьми также чаще выплачиют кредиты.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [44]:
debt_from_family_status = pd.DataFrame()
debt_from_family_status['sum_family_status'] = df.groupby('family_status')['debt'].sum()
debt_from_family_status['count_family_status'] = df.groupby('family_status')['debt'].count()
debt_from_family_status['result_family_status'] = debt_from_family_status['sum_family_status'] / debt_from_family_status['count_family_status'] 
debt_from_family_status.sort_values('result_family_status', ascending = False)

,sum_family_status,count_family_status,result_family_status
family_status,,,
не женат / не замужем,274,2810,0.097509
гражданский брак,388,4150,0.093494
женат / замужем,931,12339,0.075452
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693


**Вывод** Заемщики, которые не были в браке имеют больше просрочек по кредитам. Те же, которые ранее состояли в браке выплачивают кредиты лучше тех, кто сейчас в браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [45]:
count_total_income_status = df.groupby('total_income_status')['debt'].count()
sum_total_income_status = df.groupby('total_income_status')['debt'].sum()
debt_total_income_status = sum_total_income_status / count_total_income_status
debt_total_income_status

total_income_status
высокий          0.068990
выше среднего    0.084128
низкий           0.061828
средний          0.082420
Name: debt, dtype: float64

**Вывод** Как ни странно лучшими заемщиками являются люди с низким и высоким доходом. Люди со средним доходом чаще совершают просрочки по кредитам.

- Как разные цели кредита влияют на его возврат в срок?

In [46]:
count_purpose_loan = df.groupby('purpose_loan')['debt'].count()
sum_purpose_loan = df.groupby('purpose_loan')['debt'].sum()
debt_purpose_loan = sum_purpose_loan / count_purpose_loan
debt_purpose_loan

purpose_loan
авто            0.093590
недвижимость    0.072334
образование     0.092200
свадьба         0.080069
Name: debt, dtype: float64

**Вывод** Самые надежные заемщики те, кто приобретает недвижимость. Чаще всех просрочки случаются у тех, кто берет авто в кредит.

## Шаг 4. Общий вывод

In [47]:
lonely_count = df[df['family_status'] == 'не женат / не замужем']['debt'].count()
lonely_sum = df[df['family_status'] == 'не женат / не замужем']['debt'].sum()

divorce_count = df[df['family_status'] == 'в разводе']['debt'].count()
divorce_sum = df[df['family_status'] == 'в разводе']['debt'].sum()

widow_count = df[df['family_status'] == 'вдовец / вдова']['debt'].count()
widow_sum = df[df['family_status'] == 'вдовец / вдова']['debt'].sum()

live_together_count = df[df['family_status'] == 'гражданский брак']['debt'].count()
live_together_sum = df[df['family_status'] == 'гражданский брак']['debt'].sum()

married_count = df[df['family_status'] == 'женат / замужем']['debt'].count()
married_sum = df[df['family_status'] == 'женат / замужем']['debt'].sum()

print('Доля одиноких с задолженностью : {:.2%}'.format(lonely_sum / lonely_count))
print('Доля разведенных с задолженностью : {:.2%}'.format(divorce_sum / divorce_count))
print('Доля вдовцов и вдов с задолженностью : {:.2%}'.format(widow_sum / widow_count))
print('Доля состоящих в гражданском браке с задолженностью : {:.2%}'.format(live_together_sum / live_together_count))
print('Доля состоящих в браке с задолженностью : {:.2%}'.format(married_sum / married_count))



Доля одиноких с задолженностью : 9.75%
Доля разведенных с задолженностью : 7.11%
Доля вдовцов и вдов с задолженностью : 6.57%
Доля состоящих в гражданском браке с задолженностью : 9.35%
Доля состоящих в браке с задолженностью : 7.55%


In [48]:
childless_count = df[df['children'] == 0]['debt'].count()
childless_sum = df[df['children'] == 0]['debt'].sum()

children_1_count = df[df['children'] == 1]['debt'].count()
children_1_sum = df[df['children'] == 1]['debt'].sum()

children_2_count = df[df['children'] == 2]['debt'].count()
children_2_sum = df[df['children'] == 2]['debt'].sum()

children_3_count = df[df['children'] == 3]['debt'].count()
children_3_sum = df[df['children'] == 3]['debt'].sum()

children_4_count = df[df['children'] == 4]['debt'].count()
children_4_sum = df[df['children'] == 4]['debt'].sum()

children_5_count = df[df['children'] == 5]['debt'].count()
children_5_sum = df[df['children'] == 5]['debt'].sum()


print('Доля бездетных с задолженностью : {:.2%}'.format(childless_sum / childless_count))
print('Доля заемщиков с 1 ребенком с задолжностью : {:.2%}'.format(children_1_sum / children_1_count))
print('Доля заемщиков с 2 детьми с задолжностью : {:.2%}'.format(children_2_sum / children_2_count))
print('Доля заемщиков с 3 детьми с задолжностью : {:.2%}'.format(children_3_sum / children_3_count))
print('Доля заемщиков с 4 детьми с задолжностью : {:.2%}'.format(children_4_sum / children_4_count))
print('Доля заемщиков с 5 детьми с задолжностью : {:.2%}'.format(children_5_sum / children_5_count))

Доля бездетных с задолженностью : 7.56%
Доля заемщиков с 1 ребенком с задолжностью : 9.17%
Доля заемщиков с 2 детьми с задолжностью : 9.45%
Доля заемщиков с 3 детьми с задолжностью : 8.18%
Доля заемщиков с 4 детьми с задолжностью : 9.76%
Доля заемщиков с 5 детьми с задолжностью : 0.00%


In [49]:
low_income_count = df[df['total_income_status'] == 'низкий']['debt'].count()
low_income_sum = df[df['total_income_status'] == 'низкий']['debt'].sum()

average_income_count = df[df['total_income_status'] == 'средний']['debt'].count()
average_income_sum = df[df['total_income_status'] == 'средний']['debt'].sum()

above_average_income_count = df[df['total_income_status'] == 'выше среднего']['debt'].count()
above_average_income_sum = df[df['total_income_status'] == 'выше среднего']['debt'].sum()

high_income_count = df[df['total_income_status'] == 'высокий']['debt'].count()
high_income_sum = df[df['total_income_status'] == 'высокий']['debt'].sum()

print('Доля заемщиков с низким доходом с задолженностью : {:.2%}'.format(low_income_sum / low_income_count))
print('Доля заемщиков со средним доходом с задолженностью : {:.2%}'.format(average_income_sum / average_income_count))
print('Доля заемщиков с доходом выше среднего с задолженностью : {:.2%}'.format(above_average_income_sum / above_average_income_count))
print('Доля заемщиков с высоким доходом с задолженностью : {:.2%}'.format(high_income_sum / high_income_count))

Доля заемщиков с низким доходом с задолженностью : 6.18%
Доля заемщиков со средним доходом с задолженностью : 8.24%
Доля заемщиков с доходом выше среднего с задолженностью : 8.41%
Доля заемщиков с высоким доходом с задолженностью : 6.90%


In [50]:
auto_credit_count = df[df['purpose_loan'] == 'авто']['debt'].count()
auto_credit_sum = df[df['purpose_loan'] == 'авто']['debt'].sum()

estate_credit_count = df[df['purpose_loan'] == 'недвижимость']['debt'].count()
estate_credit_sum = df[df['purpose_loan'] == 'недвижимость']['debt'].sum()

married_credit_count = df[df['purpose_loan'] == 'свадьба']['debt'].count()
married_credit_sum = df[df['purpose_loan'] == 'свадьба']['debt'].sum()

education_credit_count = df[df['purpose_loan'] == 'образование']['debt'].count()
education_credit_sum = df[df['purpose_loan'] == 'образование']['debt'].sum()

print('Доля заемщиков с целью приобретения авто с задолженностью : {:.2%}'.format(auto_credit_sum / auto_credit_count))
print('Доля заемщиков с целью приобретения недвижимости с задолженностью : {:.2%}'.format(estate_credit_sum / estate_credit_count))
print('Доля заемщиков с целью оплаты свадьбы с задолженностью : {:.2%}'.format(married_credit_sum / married_credit_count))
print('Доля заемщиков с целью оплаты образования с задолженностью : {:.2%}'.format(education_credit_sum / education_credit_count))


Доля заемщиков с целью приобретения авто с задолженностью : 9.36%
Доля заемщиков с целью приобретения недвижимости с задолженностью : 7.23%
Доля заемщиков с целью оплаты свадьбы с задолженностью : 8.01%
Доля заемщиков с целью оплаты образования с задолженностью : 9.22%


**Вывод** Более надежными заемщиками будут одинокие люди, состоявшие ранее в браке. Самые надежные заемщики те, кто планируют приобретать недвижимость. Заемщики со средним доходом чаще совершают прострочки. Бездетные люди более дисциплинированны. С увеличением количества детей растет и доля просрочек.